In [ ]:
!pip install librosa --upgrade

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import IPython.display as ipd
import copy
import time

!gdown https://drive.google.com/uc?id=110yz4UGIbveLW8ENFI1DjypNXs96q0FZ

y, sr = librosa.load('/content/ex.wav',sr=16000) #Читаем файл
ipd.Audio(y, rate=sr) 

Синетез речи не является простой задачей, так как звук содержит много данных, которые имеют сложные и долгосрочные зависимости. Из-за этого задачу разбивают на две. Сначала синтезатор из текста получают спектр, или мелспектр. Потом вокодер из него уже создаёт звук. 

Следующий блок объясняет, почему нельзя просто использовать обратное преобразование Фурье для получения сигнала только из амплитудного спектра, и почему сложно предсказывать фазу вместе с амплитудой

# Сказ о том, почему сложно отказаться от вокодера

In [3]:
class Hparams():
  def __init__(self, sr):
    self.n_fft = int(sr * 25 / 1000)
    self.hop_length=int(sr * 10 / 1000)
    self.n_mels = 40
    self.n_iter = 10
    self.win_length = 20

## Получить  спектр

In [ ]:
hp = Hparams(sr)
# Получаем спектр
spectr = librosa.stft(y=y,hop_length=hp.hop_length,n_fft=hp.n_fft)
# Амплитуда
ampl = np.abs(spectr) 
# Фаза
phasa = np.angle(spectr)
# Исходный сигнал только по амплитуде
y_ampl = librosa.istft(ampl,hop_length=hp.hop_length)
ipd.Audio(y_ampl, rate=sr) 

Можно заменить, что звук, восстановленный только по амплитуде, очень некачественный

## Вернуть фазу

In [ ]:
# Добавляем фазу
ampl_phasa =  ampl*np.cos(phasa) + 1j*ampl*np.sin(phasa)
# Исходный сигнал по амплитуде и фазе
y_ampl_phasa = librosa.istft(ampl_phasa,hop_length=hp.hop_length)
ipd.Audio(y_ampl_phasa, rate=sr) 

Если же вернуть фазу, то всё ок

## Получить мелспектрограмму

In [ ]:
melspectr = librosa.feature.melspectrogram( #Мелспектр
    y,
    sr,
    n_fft=hp.n_fft,
    hop_length=hp.hop_length,
    n_mels=hp.n_mels
)

#Для отображения
S_dB = librosa.power_to_db(melspectr, ref=np.max)
plt.figure(figsize=(10, 4))
cmap = cm.get_cmap('viridis')
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', cmap=cmap)
plt.colorbar()
plt.title('Melspectrogram')
plt.tight_layout()
plt.show()

## Мелспектрограмма в аудио

In [ ]:
ampl_mel = librosa.feature.inverse.mel_to_stft(melspectr, sr, n_fft=hp.n_fft)
y_mel_ampl = librosa.istft(ampl_mel,hop_length=hp.hop_length)
ipd.Audio(y_mel_ampl, rate=sr) 

Звук по амплитуде, полученной по мел-сигналу, тоже плохой

## Гриффин-Лим

In [ ]:
y_griffinlim = librosa.griffinlim(ampl_mel, hop_length=hp.hop_length)
ipd.Audio(y_griffinlim, rate=sr) 

Данный алгоритм пытается восстановить фазу, что улучшает качество

## Вернуть к мел фазу

In [ ]:
ampl_phasa =  ampl_mel*np.cos(phasa) + 1j*ampl_mel*np.sin(phasa)
y_mel_ampl_phasa = librosa.istft(ampl_phasa,hop_length=hp.hop_length)
ipd.Audio(y_mel_ampl_phasa, rate=sr) 

Если же восстановить сигнал, добавив исходную фазу, то снова всё хорошо. А значит переход в мелспектр не сильно ухудшает качество.

## Почему мы в синтезаторе отбрасываем фазу

### Амплитуда несёт информацию

In [ ]:
Xdb = 10.0 * np.log10(ampl**2 + 1e-10)
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

### Фаза случайна

In [ ]:
librosa.display.specshow(phasa, sr=sr, x_axis='time', y_axis='hz')

В фазе нет паттерна, который нейронная сеть может хорошо достать

## Отобразить все графики

In [ ]:
plt.figure(figsize=(16, 12))
ax = plt.subplot(5,1,1)
librosa.display.waveplot(y, sr=sr, color='b')
plt.title('Оригинал')
plt.xlabel('')
plt.subplot(5,1,2, sharex=ax, sharey=ax)
librosa.display.waveplot(y_ampl, sr=sr, color='r')
plt.title('Только Амплитуда')
plt.xlabel('')
plt.subplot(5,1,3, sharex=ax, sharey=ax)
librosa.display.waveplot(y_mel_ampl, sr=sr, color='r')
plt.title('Только Амплитуда, полученная из Mel')
plt.xlabel('')
plt.subplot(5,1,4, sharex=ax, sharey=ax)
librosa.display.waveplot(y_griffinlim, sr=sr, color='g')
plt.title('Только Амплитуда, полученная из Mel + griffinlim')
plt.xlabel('')
plt.subplot(5,1,5, sharex=ax, sharey=ax)
librosa.display.waveplot(y_mel_ampl_phasa, sr=sr, color='r')
plt.title('Амплитуда и фаза, полученная из Mel')
plt.tight_layout()
plt.show()

## Выводы

Хотя основная информация содержится в амплитуде, без информации о фазе восстановить качественный сигнал не получается. Можно пробовать использовать алгоритм Гриффин-Лима, но и он не даёт качественного сигнала. 

В силу своей природы амплитудный спектр содержит много характерных участков, которые сеть может выучить и восстановить. Фазовый спектр напротив, очень случаен и сложно восстановим.

Именно поэтому существующие алгоритмы сначала получают спектр, а уже потом из него получают сигнал, без предсказывания фазы.


# MelGan

Существующие модели, в основном, авторегресионные, а значит куски звука генерируются последовательно, а значит скорость их работы не очень высока.

Авторы статьи решили использовать сеть GAN, которые до этого обычно использовались для работы с картинками, для преоброзования мелспектра в сырой звук. 

Архитектура состоит из генератора MelGAN, из 4 транспонированных сверток и 3*4 Residual блоков. А также трёх дискрименаторов, каждый из которых содержит 4 понижающие свёртки. Второй и третий дискрименатор принимает сигнал сжатый в 2 и 4 раза соответсвенно, что позволяет больше внимания уделять нижним частотам. 

In [ ]:
!git clone https://github.com/vlomme/MelGan-WavGan.git

In [ ]:
cd MelGan-WavGan/

In [ ]:
!gdown https://drive.google.com/uc?id=10tLduS5fGNWby7IKvfltuIfUWUeAp9SM
!unzip logs.zip

## Синтез из мелспектрограмм

In [ ]:
!python melgan.py -r g -f mel

### Оригинальный сигнал

In [ ]:
y_orig, sr = librosa.load('./gen/orig/000000_RUSLAN.wav',sr=16000)
ipd.Audio(y_orig, rate=sr) 

### Вокодер MelGan

In [ ]:
y_melgan, sr = librosa.load('./gen/output/mel_000000_RUSLAN.wav',sr=16000)
ipd.Audio(y_melgan, rate=sr) 

## Синтез из Гриффин-Лим сигнала

Если же ей на вход подавать мелспектр, полученный из Гриффин-Лим сигнала, то качество будет хуже

In [ ]:
!python melgan.py -r g -f wav

### Оригинальный сигнал

In [ ]:
y_orig, sr = librosa.load('./gen/orig/RUSLAN.wav',sr=16000)
ipd.Audio(y_orig, rate=sr) 

### Гриффин Лим

In [ ]:
y_griffinlim, sr = librosa.load('./gen/RUSLAN.wav',sr=16000)
ipd.Audio(y_griffinlim, rate=sr) 

### Вокодер MelGan

In [ ]:
y_melgan, sr = librosa.load('./gen/output/mel_RUSLAN.wav',sr=16000)
ipd.Audio(y_melgan, rate=sr) 

### Выводы

Мои эксперименты показали, что модель даёт хорошее качество, чуть хуже, чем WaveRNN, но работает существенно быстрее. Для улучшения качества требуется либо обучать модель существенно дольше, или пробовать другие подходы.

Из недостатков можно заметить, что модель требует мелспектрограммы в том формате, на котором она обучалась, что требует согласования и переобучения под конкретные модели синтезатора. 


# WavGan

Идея. Зачем синтезировать из мелспектрограммы, если можно применить Гриффин-Лим и получить сигнал, очень похожий на оригинальный звук и останется его только подправить. Данный подход привёл к разработке WavGan. Структура его генератора похожа на U-net, а дискриминатор взят из MelGAN. Такая модель существенно быстрее учится, не требует настройки под различные синтезаторы, потенциально может удалять шумы и даёт схожее с MelGAN качество.

## Синтез из мелспектрограмм

In [ ]:
!python wavgan.py -r g -f mel

In [ ]:
y_wavgan, sr = librosa.load('./gen/output/wav_000000_RUSLAN.wav',sr=16000)
ipd.Audio(y_wavgan, rate=sr) 

## Синтез из Гриффин-Лим сигнала

In [ ]:
!python wavgan.py -r g -f wav

In [ ]:
y_wavgan, sr = librosa.load('./gen/output/wav_RUSLAN.wav',sr=16000)
ipd.Audio(y_wavgan, rate=sr) 

# TTS

Здесь показан подход TTS. В качестве синтезатора используется Tacotron 2, а в качестве вокодера WavGAN

In [ ]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!sudo apt-get install portaudio19-dev
!pip install PyAudio
!pip install sounddevice
!pip install unidecode

!gdown https://drive.google.com/uc?id=19Gl9ejMUm_ll6cKozt5N99qDBN25FXDA
!unzip test.zip

In [ ]:
cd test/

In [ ]:
!python synthesize.py --text_list 'Привет сотрудники вконтакте. Спасибо, что дочитали до этого места' --checkpoint male
#!python synthesize.py --text_list 'Привет сотрудники вконтакте. Спасибо, что дочитали до этого места' --checkpoint female
#!python synthesize.py --text_list 'Привет сотрудники вконтакте. Спасибо, что дочитали до этого места' --checkpoint zueva

In [ ]:
y_tac, sr = librosa.load('./tacotron_output/wav-batch_0_sentence_0-mel.wav',sr=16000)
ipd.Audio(y_tac, rate=sr) 

In [ ]:
!python wavgan.py -r g -f wav

In [ ]:
y_tac, sr = librosa.load('./tacotron_output/output/wav_wav-batch_0_sentence_0-mel.wav',sr=16000)
ipd.Audio(y_tac, rate=sr) 